In [1]:
import evaluate
import numpy as np
from datasets import load_from_disk, disable_caching
from sklearn.metrics import f1_score
from transformers import (
    AutoTokenizer,
    RobertaForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [2]:
# import random
# seeds = [random.randint(0, 1e9) for _ in range(5)]
# seeds

In [3]:
disable_caching()

In [4]:
num_labels = 5
id2label = {
    0: "reject",
    1: "B_supplies_A",
    2: "A_supplies_B",
    3: "ambiguous",
    4: "ownership",
}
label2id = {
    "reject": 0,
    "B_supplies_A": 1,
    "A_supplies_B": 2,
    "ambiguous": 3,
    "ownership": 4,
}
metric = evaluate.load("f1")

In [5]:
model_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["__NE_FROM__", "__NE_TO__", "__NE_OTHER__"]}
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_micro = f1_score(labels, predictions, average='micro')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_classwise = f1_score(labels, predictions, average=None)

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        **{f"f1_class_{i}": score for i, score in enumerate(f1_classwise)}
    }


def model_init():
    model = RobertaForSequenceClassification.from_pretrained(
        model_name,
        num_labels=5,
        id2label=id2label,
        label2id=label2id,
    )
    model.resize_token_embeddings(len(tokenizer))
    return model

In [7]:
ds = load_from_disk("../../datasets/ManualDataset")
ds = ds.select_columns(["masked_text", "label"])
ds = ds.rename_column("masked_text", "text")
ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2559 [00:00<?, ? examples/s]

Map:   0%|          | 0/418 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [8]:
def run_experiment(seed):
    set_seed(seed)
    training_args = TrainingArguments(
        seed=seed,
        data_seed=seed,
        tf32=True,
        output_dir="logs/experiment_1_roberta",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        warmup_ratio=0.1,
        load_best_model_at_end=True,
        save_total_limit=1,
        report_to=[],
        save_only_model=True,
    )
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["valid"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    test_results = trainer.predict(ds["test"])
    return test_results

In [9]:
seeds = [87429541, 201431458, 584301348, 973765745, 499095391]
all_results = []

for seed in seeds:
    results = run_experiment(seed)
    all_results.append(results)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 1.1323809623718262, 'eval_f1_micro': 0.5933014354066986, 'eval_f1_macro': 0.40542614200648286, 'eval_f1_class_0': 0.6993318485523385, 'eval_f1_class_1': 0.0, 'eval_f1_class_2': 0.5294117647058824, 'eval_f1_class_3': 0.5483870967741935, 'eval_f1_class_4': 0.25, 'eval_runtime': 0.362, 'eval_samples_per_second': 1154.743, 'eval_steps_per_second': 74.589, 'epoch': 1.0}


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.550137996673584, 'eval_f1_micro': 0.8277511961722488, 'eval_f1_macro': 0.8010323384851574, 'eval_f1_class_0': 0.8308605341246291, 'eval_f1_class_1': 0.6857142857142857, 'eval_f1_class_2': 0.8725868725868726, 'eval_f1_class_3': 0.816, 'eval_f1_class_4': 0.8, 'eval_runtime': 0.5883, 'eval_samples_per_second': 710.481, 'eval_steps_per_second': 45.892, 'epoch': 9.0}
{'loss': 0.307, 'grad_norm': 25.95094108581543, 'learning_rate': 1.3888888888888892e-06, 'epoch': 9.38}


  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.5589311122894287, 'eval_f1_micro': 0.8277511961722488, 'eval_f1_macro': 0.8014969087540427, 'eval_f1_class_0': 0.8294117647058824, 'eval_f1_class_1': 0.6956521739130435, 'eval_f1_class_2': 0.875968992248062, 'eval_f1_class_3': 0.8064516129032258, 'eval_f1_class_4': 0.8, 'eval_runtime': 0.3957, 'eval_samples_per_second': 1056.274, 'eval_steps_per_second': 68.228, 'epoch': 10.0}
{'train_runtime': 99.5721, 'train_samples_per_second': 257.0, 'train_steps_per_second': 16.069, 'train_loss': 0.629484829902649, 'epoch': 10.0}


  0%|          | 0/47 [00:00<?, ?it/s]

In [10]:
# Calculate mean and std of F1 scores
metrics = ["test_f1_micro", "test_f1_macro"] + [f"test_f1_class_{i}" for i in range(5)]  # Adjust range based on num_classes

avg_results = {}
for metric in metrics:
    scores = [r[metric] for r in [x.metrics for x in all_results]]
    avg_results[metric] = {
        'mean': np.mean(scores),
        'std': np.std(scores)
    }

# Print results
for metric in metrics:
    print(f"Average {metric}: {avg_results[metric]['mean']:.4f} ± {avg_results[metric]['std']:.4f}")

Average test_f1_micro: 0.7557 ± 0.0318
Average test_f1_macro: 0.7458 ± 0.0331
Average test_f1_class_0: 0.7687 ± 0.0411
Average test_f1_class_1: 0.7384 ± 0.0281
Average test_f1_class_2: 0.7892 ± 0.0379
Average test_f1_class_3: 0.6426 ± 0.0286
Average test_f1_class_4: 0.7902 ± 0.0552
